In [88]:
### download and extract chinook sample DB
import urllib.request
import zipfile
from functools import partial
import os

chinook_url = 'http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip'
if not os.path.exists('chinook.zip'):
    print('downloading chinook.zip ', end='')
    with urllib.request.urlopen(chinook_url) as response:
        with open('chinook.zip', 'wb') as f:
            for data in iter(partial(response.read, 4*1024), b''):
                print('.', end='', flush=True)
                f.write(data)

zipfile.ZipFile('chinook.zip').extractall()
assert os.path.exists('chinook.db')

In [89]:
### useful: functions for displaying results from sql queries using pandas
from IPython.display import display
import pandas as pd

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)

# Exercise 1 : Open the database

In [90]:
# open the database using sqlalchemy module interface
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, select

# create an engine object in a variable named engine
engine = create_engine("sqlite:///chinook.db")

# establishing connection to engine
conn = engine.connect()

In [91]:
### useful: extract classes from the chinook database
metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

## we need to do this once
from sqlalchemy.ext.automap import automap_base

# produce a set of mappings from this MetaData.
Base = automap_base(metadata=metadata)

# calling prepare() just sets up mapped classes and relationships.
Base.prepare()

# also prepare an orm session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

# Exercise 2: Table Names

In [92]:
# Print the Name of Each Table in the Database
print(metadata.tables.keys())

# OR Print Table Names + Columns
for table_name, orm_class in Base.classes.items():
    cols = ", ".join([col.name for col in orm_class.__table__.columns])
    print(f"{table_name}: {cols}")

dict_keys(['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'tracks', 'media_types', 'invoices', 'playlist_track', 'playlists'])
genres: GenreId, Name
artists: ArtistId, Name
albums: AlbumId, Title, ArtistId
playlists: PlaylistId, Name
employees: EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email
invoices: InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total
invoice_items: InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity
tracks: TrackId, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice
media_types: MediaTypeId, Name
customers: CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId


# Exercise 3: Tracks 

In [93]:
from sqlalchemy.sql import text
from sqlalchemy import select

# Check the name of every column in Tracks Table
tracks = metadata.tables['tracks']
# print([c.name for c in tracks.columns])

# Select the First 3 Rows from Table
query = select(tracks.c.Name)
exe = conn.execute(query)
result = exe.fetchmany(3)

# Print the Tracks Row by Row
for row in result:
    print(row[0])

# OR - Using Text Method
query = text("""
SELECT Name FROM tracks LIMIT 3
""")
exe = conn.execute(query)

result = exe.fetchall()
for row in result:
    print(row[0])



For Those About To Rock (We Salute You)
Balls to the Wall
Fast As a Shark
For Those About To Rock (We Salute You)
Balls to the Wall
Fast As a Shark


# Exercise 4 : Albums from Tracks

In [94]:
# print out the track name and albums title of the first 20 tracks in the tracks table
# create instance for albumns table
albums = metadata.tables['albums']

# Join the tables to be able to Retrieve the Album Title
query = (
    select(tracks.c.Name, albums.c.Title)
    .select_from(tracks.join(albums, tracks.c.AlbumId == albums.c.AlbumId))
)

# Execute Query and Fetch the Result
exe = conn.execute(query)
result = exe.fetchmany(20)

# Print the result row by row
for row in result:
    print(f"{row[0]} - {row[1]}")


For Those About To Rock (We Salute You) - For Those About To Rock We Salute You
Put The Finger On You - For Those About To Rock We Salute You
Let's Get It Up - For Those About To Rock We Salute You
Inject The Venom - For Those About To Rock We Salute You
Snowballed - For Those About To Rock We Salute You
Evil Walks - For Those About To Rock We Salute You
C.O.D. - For Those About To Rock We Salute You
Breaking The Rules - For Those About To Rock We Salute You
Night Of The Long Knives - For Those About To Rock We Salute You
Spellbound - For Those About To Rock We Salute You
Balls to the Wall - Balls to the Wall
Fast As a Shark - Restless and Wild
Restless and Wild - Restless and Wild
Princess of the Dawn - Restless and Wild
Go Down - Let There Be Rock
Dog Eat Dog - Let There Be Rock
Let There Be Rock - Let There Be Rock
Bad Boy Boogie - Let There Be Rock
Problem Child - Let There Be Rock
Overdose - Let There Be Rock


In [95]:
# Using Text Method
query = text("""
             SELECT t.Name, a.Title
             FROM tracks AS t
             JOIN albums AS a
             ON t.AlbumID == a.AlbumID
             """)

exe = conn.execute(query)
result = exe.fetchmany(20)

for row in result:
    print(f"{row[0]} - {row[1]}")

For Those About To Rock (We Salute You) - For Those About To Rock We Salute You
Put The Finger On You - For Those About To Rock We Salute You
Let's Get It Up - For Those About To Rock We Salute You
Inject The Venom - For Those About To Rock We Salute You
Snowballed - For Those About To Rock We Salute You
Evil Walks - For Those About To Rock We Salute You
C.O.D. - For Those About To Rock We Salute You
Breaking The Rules - For Those About To Rock We Salute You
Night Of The Long Knives - For Those About To Rock We Salute You
Spellbound - For Those About To Rock We Salute You
Balls to the Wall - Balls to the Wall
Fast As a Shark - Restless and Wild
Restless and Wild - Restless and Wild
Princess of the Dawn - Restless and Wild
Go Down - Let There Be Rock
Dog Eat Dog - Let There Be Rock
Let There Be Rock - Let There Be Rock
Bad Boy Boogie - Let There Be Rock
Problem Child - Let There Be Rock
Overdose - Let There Be Rock


# Exercise 5: Tracks sold

In [96]:
# print out the first 10 track sales from the invoice_items table
query = text("""SELECT * FROM invoice_items LIMIT 10""")
exe = conn.execute(query)
result = exe.fetchall()

for row in result:
    print(row)

(1, 1, 2, 0.99, 1)
(2, 1, 4, 0.99, 1)
(3, 2, 6, 0.99, 1)
(4, 2, 8, 0.99, 1)
(5, 2, 10, 0.99, 1)
(6, 2, 12, 0.99, 1)
(7, 3, 16, 0.99, 1)
(8, 3, 20, 0.99, 1)
(9, 3, 24, 0.99, 1)
(10, 3, 28, 0.99, 1)


In [97]:
# Print the Names of the Track Sold and the Quantity Sold
query = text("""
SELECT tracks.Name, invoice_items.Quantity
FROM invoice_items
JOIN tracks
ON tracks.TrackId = invoice_items.TrackId
""")

# Execute Above Query
exe = conn.execute(query)
result = exe.fetchmany(10)

# Print Out the Result
for row in result:
    print(f"Track: {row[0]}\nQuantity: {row[1]}\n")


Track: Balls to the Wall
Quantity: 1

Track: Restless and Wild
Quantity: 1

Track: Put The Finger On You
Quantity: 1

Track: Inject The Venom
Quantity: 1

Track: Evil Walks
Quantity: 1

Track: Breaking The Rules
Quantity: 1

Track: Dog Eat Dog
Quantity: 1

Track: Overdose
Quantity: 1

Track: Love In An Elevator
Quantity: 1

Track: Janie's Got A Gun
Quantity: 1



# Exercise 6 : Top tracks sold

In [98]:
# print the names of top 10 tracks sold, and how many they times they were sold
query = text("""
SELECT t.Name, sum(i.Quantity) AS QuantitySold
FROM invoice_items AS i
JOIN tracks as t
ON i.TrackID = t.TrackID
GROUP BY i.TrackID, t.Name
ORDER BY QuantitySold DESC
""")

exe = conn.execute(query)
result = exe.fetchmany(10)

for row in result:
    print(f"Track: {row[0]} - Quantity Sold: {row[1]}")


Track: Balls to the Wall - Quantity Sold: 2
Track: Inject The Venom - Quantity Sold: 2
Track: Snowballed - Quantity Sold: 2
Track: Overdose - Quantity Sold: 2
Track: Deuces Are Wild - Quantity Sold: 2
Track: Not The Doctor - Quantity Sold: 2
Track: Por Causa De Você - Quantity Sold: 2
Track: Welcome Home (Sanitarium) - Quantity Sold: 2
Track: Snowblind - Quantity Sold: 2
Track: Cornucopia - Quantity Sold: 2


# Exercise 7: Exercise 7 : Top selling artists

In [99]:
# Who are the top 10 highest selling artists?
query = text("""
SELECT a.Name, SUM(i.Quantity * i.UnitPrice) AS Sales
FROM invoice_items AS i
JOIN tracks AS t ON i.TrackID = t.TrackID
JOIN albums AS al ON t.AlbumID = al.AlbumId
JOIN artists AS a ON al.ArtistID = a.ArtistID
GROUP BY a.Name
ORDER BY Sales DESC
""")

exe = conn.execute(query)
result = exe.fetchmany(10)

for row in result:
    print(f"Artist: {row[0]}\nTotal Sales: ${row[1]:.2f}\n")

Artist: Iron Maiden
Total Sales: $138.60

Artist: U2
Total Sales: $105.93

Artist: Metallica
Total Sales: $90.09

Artist: Led Zeppelin
Total Sales: $86.13

Artist: Lost
Total Sales: $81.59

Artist: The Office
Total Sales: $49.75

Artist: Os Paralamas Do Sucesso
Total Sales: $44.55

Artist: Deep Purple
Total Sales: $43.56

Artist: Faith No More
Total Sales: $41.58

Artist: Eric Clapton
Total Sales: $39.60

